In [2]:
import pandas as pd
import numpy as np
import os 
import geopandas as gpd
import plotly.express as px

In [7]:
mf = gpd.read_file('./Marine_forest/Marine_forest.shp')
count = len(mf[(mf['면적'] == mf['구역']) & (mf['면적'] == mf['규모'])])
print(count)
mf.drop(columns=['구역','규모'],inplace=True)
col_names = {'시도명': 'province','시군구명':'city','시군구코드':'city_code','위도':'latitude','경도':'longitude','마을어장':'town','면적':'area'}
mf.rename(columns=col_names, inplace=True)
#기본 칼럼인 위도 경도는 오류가 있었음
mf['latitude'] = mf['geometry'].y
mf['longitude'] = mf['geometry'].x
# mf = pd.DataFrame(mf)
# mf.to_csv('./Marine_forest/mf.csv', index=False)
mf

219


,province,city,city_code,town,latitude,longitude,시설년도,시설유형,품종,area,ORIG_FID,geometry
0,충청남도,서산시,21,가로림만1단지,36.918893,126.335300,2013,어초,감태,3.640241e+05,0,POINT (126.33530 36.91889)
1,충청남도,서산시,21,가로림만2단지,36.910919,126.353197,2013,어초,감태,2.338867e+05,1,POINT (126.35320 36.91092)
2,충청남도,서산시,21,가로림만3단지,36.896696,126.348674,2013,어초,감태,4.164415e+05,2,POINT (126.34867 36.89670)
3,강원도,고성군,82,가진리,38.368750,128.518750,2018,어초,미역,1.514584e+06,3,POINT (128.51875 38.36875)
4,강원도,삼척시,23,갈남리,37.281873,129.329315,2012,어초,미역,3.635963e+05,4,POINT (129.32931 37.28187)
...,...,...,...,...,...,...,...,...,...,...,...,...
214,강원도,고성군,82,고성군 청간리,38.268750,128.568750,2022,어초,미역,1.516629e+06,214,POINT (128.56875 38.26875)
215,강원도,속초시,21,속초시 대포동,38.168750,128.618750,2022,어초,감태,1.518669e+06,215,POINT (128.61875 38.16875)
216,경상북도,포항시,11,포항시 하정2리,35.968426,129.557133,2022,어초,미역,1.308860e+06,216,POINT (129.55713 35.96843)
217,경상북도,포항시,11,포항시 계원1리,35.865283,129.534375,2022,어초,감태,1.025338e+06,217,POINT (129.53438 35.86528)


In [8]:
# mf_2013 = pd.DataFrame(mf.loc[mf['시설년도']<= 2013])
# mf_2014 = pd.DataFrame(mf.loc[mf['시설년도']<= 2014])
# mf_2015 = pd.DataFrame(mf.loc[mf['시설년도']<= 2015])
# mf_2016 = pd.DataFrame(mf.loc[mf['시설년도']<= 2016])
# mf_2017 = pd.DataFrame(mf.loc[mf['시설년도']<= 2017])
# mf_2018 = pd.DataFrame(mf.loc[mf['시설년도']<= 2018])
# mf_2019 = pd.DataFrame(mf.loc[mf['시설년도']<= 2019])
# mf_2020 = pd.DataFrame(mf.loc[mf['시설년도']<= 2020])

전체에 적용할 기본적인 전처리

In [32]:
# Ocean_data 폴더 내의 모든 CSV 파일 목록을 가져옵니다.
folder_path = 'Ocean_data'
all_files = [f for f in os.listdir(folder_path) if f.endswith('.csv') and f.startswith('19') or f.startswith('20')]
# 제거하려는 열의 리스트
cols_to_drop = ['dep_flag', '유효시간']
# 각 파일을 순차적으로 읽어 데이터 프레임 리스트에 추가합니다.
dfs = []
for filename in all_files:
    file_path = os.path.join(folder_path, filename)
    df = pd.read_csv(file_path)
    ##Longitude:위도 , Latitude: 경도
    colnames = {'yyyy-mm-dd hh:mm:ss': '일시', 'Longitude [degrees_east]': 'longitude', 'Latitude [degrees_north]': 'latitude'}
    df.rename(columns=colnames, inplace=True)

    data = df.copy()
    #데이터 품질값이 1~3인 값만 남기고 나머지는 NaN으로 대체합니다.
    qc_cols = [(idx, col) for idx, col in enumerate(data.columns) if col.endswith('_qc')]
    for idx, col in qc_cols:
        obs_df = data.iloc[:, [idx - 1, idx]].copy()
        obs_df.iloc[~obs_df.iloc[:, 1].isin([1, 2,3]), 0] = np.nan
        col_value = data.columns[idx - 1]
        data[col_value] = obs_df.iloc[:, 0]
    #qc칼럼 제거
    qc_cols = [col for col in data.columns if col.endswith('_qc')]
    data.drop(columns=qc_cols, inplace=True)

    data['일시'] = pd.to_datetime(data['일시'],format='mixed')
    data['year'] = data['일시'].dt.year
    data['month'] = data['일시'].dt.month
    data['quarter'] = data['일시'].dt.quarter
    data['year_quarter'] = data['year'].astype(str) + data['quarter'].astype(str)
    # 데이터프레임에 해당 열이 있는지 확인하고 제거합니다.
    cols_to_drop = [col for col in cols_to_drop if col in data.columns]
    data.drop(columns=cols_to_drop, inplace=True)
    data.to_csv(f'./data/{filename[:4]}.csv',index = False)

표층데이터

In [33]:
folder_path = 'data'
all_files = [f for f in os.listdir(folder_path) if f.endswith('.csv') and f.startswith('19') or f.startswith('20')]
cols_to_drop = ['수심(m)','수심(text)','bot.depth[m]']
for filename in all_files:
    file_path = os.path.join(folder_path, filename)
    df = pd.read_csv(file_path,encoding='utf-8')
    surface_df = df.copy()
    surface_df = surface_df.loc[(surface_df['수심(m)'] == 0)]
    
    cols_to_drop = [col for col in cols_to_drop if col in surface_df.columns]
    surface_df.drop(columns=cols_to_drop, inplace=True)
    surface_df.to_csv(f'./data/surface/{filename[:4]}_surface_df.csv',index= False)

바다숲과 가까이 있는 해양데이터포인터 찾기

In [ ]:
from scipy.spatial import KDTree

# 특정 반경 (예: 0.01도)
radius = 0.01
surface_df = pd.read_csv('./data/surface/2018_surface_df.csv')
# surface_df의 좌표를 KDTree에 로드
tree = KDTree(surface_df[['latitude', 'longitude']].values)

# mf의 각 좌표에 대해 주어진 반경 내의 모든 surface_df의 좌표의 인덱스를 찾습니다.
indices_list = tree.query_ball_point(mf[['latitude', 'longitude']].values, r=radius)

# 모든 인덱스를 하나의 배열로 펼칩니다.
all_indices = [idx for sublist in indices_list for idx in sublist]

# 중복 제거
unique_indices = np.unique(all_indices)

# 선택된 인덱스를 사용하여 near_df 생성
near_df = surface_df.iloc[unique_indices].reset_index(drop=True)


하나의 큰 파일 만들기

In [15]:
# Ocean_data 폴더 내의 모든 CSV 파일 목록을 가져옵니다.
folder_path = 'Ocean_data'
all_files = [f for f in os.listdir(folder_path) if f.endswith('.csv') and f.startswith('19') or f.startswith('20')]

# 각 파일을 순차적으로 읽어 데이터 프레임 리스트에 추가합니다.
dfs = []
for filename in all_files:
    file_path = os.path.join(folder_path, filename)
    df = pd.read_csv(file_path)
    dfs.append(df)

# 모든 데이터 프레임을 하나로 합칩니다.
merged_data = pd.concat(dfs, ignore_index=True)
merged_data.to_csv('./data/merged_data.csv',index = False)

In [10]:
missing_values_per_column = merged_data.isnull().sum()
print(missing_values_per_column)


station               0
일시                    0
Longitude             0
Latitude              0
수심(m)                 1
수심(text)           9536
총질소[μg/L]          3231
염분[psu]             492
수온[℃]                15
클로로필-a[μg/L]         75
부유물질 농도[μg/L]     12768
용존산소[mg/L]           35
수소이온농도[무단위]          32
규산염[μg/L]          7365
아질산성질소[μg/L]          1
인산염인[μg/L]         7619
질산성질소[μg/L]        2940
투명도[m]            15969
용존무기질소[μg/L]          1
화학적산소요구량[mg/L]        1
암모니아성 질소[μg/L]        1
유분[mg/L]          32101
총인[μg/L]              1
부유물질 농도[mg/L]     19342
year                  0
month                 0
quarter               0
year_quarter          0
dtype: int64


In [16]:
surface_df = merged_data.loc[merged_data['수심(text)'] == '표층']
surface_df.drop(columns=['수심(m)','수심(text)'],inplace=True)
surface_df.isnull().sum()
surface_df.to_csv('./data/surface_df.csv',index= False)


C:\Users\우승우\AppData\Local\Temp\ipykernel_39492\2731716758.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
low_df = merged_data.loc[merged_data['수심(text)'] == '저층']
low_df.to_csv('./data/low_df.csv',index= False)

In [13]:
columns_to_plot = ['총질소[μg/L]', '총인[μg/L]', '용존무기질소[μg/L]','화학적산소요구량[mg/L]','암모니아성 질소[μg/L]']

fig = px.scatter(low_df, x='year_quarter', y='총질소[μg/L]', title='Values over Year Quarter')
fig.show()

In [14]:
if not os.path.exists(folder_path):
    os.makedirs('./data/')
